# RTL class verification

Let's examine a bit of the internal behaviour of the RTL class. We here use some evaluation techniques that allow us to insert processing steps in between the hardware description statements during transpilation into HDL. Thus, we can apply particular verification steps according to custom design rules in the future.

During development of a variety of Signal and wire options that are potentially applicable to a RTL class hardware description, the dogma applies:

**You can never fully trust anything**

... until verified.

In [1]:
import sys
sys.path.insert(0, '../..')

from myirl.emulation.myhdl import *

In [2]:
from myirl.library import rtlclass
from myirl.emulation.factory_class import *

## Custom block example

A custom RTL block class is derived from the `genblock` class. It does not define an interface yet, so it's regarded as a kind of evaluating test bench for now, just describing a bit of mixed logic.
Note the `myhdl_worker` in MyHDL notation is decorated.

In [3]:
class MyBlock(rtlclass.genblock):
    x = 0xaa
    
    def worker(self, a, b):
        "MyIRL notation worker generator function"
        v = base.ConstSig(3, 5)
        m = a.set(a + v)
        yield [a.set(4), m, m]
     
    @factory.function
    def myhdl_worker(self, a, b):
        a.next = 0x54
        b.next = a ^ self.x
        yield delay(20)
        # b.next = (b + 1)[8:]
        b.next = b >> 3

Let's create an 'evaluator' that steps through the commands, shows the conversion result and what is calculated. First, we augment the above `MyBlock` by an `.evaluate()` method:

In [4]:
from myirl.kernel import struct_cond

class EvalBlock(MyBlock):
    def evaluate(self, proc):
        objs = []
        if isinstance(proc, list):
            for p in proc:
                gen = Evaluator(self, p, [])
                objs.append(gen)
        else:
            gen = Evaluator(self, proc, [])
            objs.append(gen)

        return objs

Then an custom evaluator is derived from the rtlclass logic generator. It overrides the `.process_logic()` method which evaluates each assignment. Note this is a simplified, incomplete version which does not deal with loops or flow control statements.

In [5]:
from myirl.targets.dummy import DummyTargetModule

class Evaluator(rtlclass.MyGen):
    def process_logic(self, gen, ctx = DummyTargetModule()):
        
        def _eval(i, ctx):
            print("| ", i)
            print("| ", i.trace_info())

            i.emit(ctx)
            if isinstance(i, base.GenAssign):
                v = i.evaluate()
                mask = ((1 << i.size()) - 1)
                h = mask & v
                print(">> (%d) `%s` = %d -- 0x%x -- %s" % (i.size(), i.destination().identifier, v, h, bin(h)))         

            print()
            
        print("======================= %s ======================" % (self.func.__name__) )
        try:
            for l in gen:
                if isinstance(l, list):
                    for i in l:
                        _eval(i, ctx)
                else:
                    _eval(l, ctx)
                    
        except TypeError as e:
            e.args = e.args + ("Possibly generator does not yield anything", )
            raise e

def dump(objs, ctx = DummyTargetModule()):
    for i in objs:
        if isinstance(i, Evaluator):
            i()
        else:
            print("OTHER", type(i))            
            

The actual logic output is occuring in `.process_logic()` which emits the HDL conversion plus the result that is expected.

The test routine again passes the signal types into the hardware description instance, here the `myhdl_worker`. The `.evaluate` method returns evaluator objects that handle each internal IRL statement.

In [6]:
import sys

def test(signed = False):
    blk = EvalBlock("eval")
    if signed:
        a, b = [ Signal(intbv()[9:].signed(), name = n) for n in "ab" ]
    else:
        a, b = [ Signal(intbv()[8:], name = n) for n in "ab" ]
        
    logic1 = blk.myhdl_worker(a, b)

    objs = blk.evaluate(logic1)
    dump(objs)

    
    return 

In [7]:
test(True)

======================= myhdl_worker ======================
|  a <= 84
|  macro::myhdl_worker:12
a <= "001010100";
>> (9) `a` = 84 -- 0x54 -- 0b1010100

|  b <= xor(a, C:170)
|  macro::myhdl_worker:13
b <= (signed(resize(a, 10)) xor "0010101010");
>> (9) `b` = 254 -- 0xfe -- 0b11111110

|  Wait: [ DeltaT 20 ns ]
|  macro::myhdl_worker:14
wait for 20 ns;

|  b <= b >> C:3
|  macro::myhdl_worker:16
b <= signed(resize(shift_right(resize(b, 9), 3), 9));
>> (9) `b` = 31 -- 0x1f -- 0b11111



## Checkpoint verification

Likewise, a simulation model in HDL can be output that is verified in a `known good` simulator against mismatches in evaluation of data type results.
Instead of automatical emission, the user is given some control.

In [8]:
from myirl.test.common_test import Checkpoint

In [9]:
from myirl import simulation as sim
from myirl.kernel.components import ComponentObj

class MyBlock2(MyBlock):
    x = 0x85
    
    check = Checkpoint
    
    @factory.function
    def myhdl_worker1(self, a, b):
        a.next = 0x54
        b.next = a ^ self.x
        self.check(b, "first check")
        b.next = (b + 1)[8:]
        b.next = b >> 3
        self.check(b, "second check")
    
    def handle_generator(self, func):
        if isinstance(func, ComponentObj):
            g = func(self)
        else:
            g = func    
        return g

    def verify(self, func, c = DummyTargetModule()):
        g = self.handle_generator(func)
        
        for gen in g:
            for stmt in gen:
                if isinstance(stmt, Checkpoint):
                    stmt()
                    stmt.logic.emit(c)
                    print("> ", stmt.trace_info())
                else:
                    stmt.emit(c)
                    stmt.evaluate()

Then run the test. The trace info from statement preceding the check is inserted:

In [10]:
def test(unit, signed = False):

    blk = unit("verify")
    if signed:
        a, b = [ Signal(intbv()[8:].signed(), name = n) for n in "ab" ]
    else:
        a, b = [ Signal(intbv()[8:], name = n) for n in "ab" ]
    logic1 = blk.myhdl_worker1(a, b)
    logic2 = blk.worker(a, b)
    
    print("====================== MYHDL process ======================")
    blk.verify(logic1)
    print("====================== MYIRL process ======================")
    blk.verify(logic2)
    
test(MyBlock2, False)

====================== MYHDL process ======================
a <= x"54";
b <= (a xor x"85");
-- Checkpoint first check {
wait for 1 ns;
print("VAL" & " " & "0x"& hstr(b));
assert (b = x"d1")
    report "first check -- expected: 209" severity failure;
-- }
>  macro::myhdl_worker1:13
b <= (resize(b, 9) + 1)(8-1 downto 0);
b <= resize(shift_right(resize(b, 8), 3), 8);
-- Checkpoint second check {
wait for 1 ns;
print("VAL" & " " & "0x"& hstr(b));
assert (b = x"1a")
    report "second check -- expected: 26" severity failure;
-- }
>  macro::myhdl_worker1:16
====================== MYIRL process ======================
a <= x"04";
a <= resize((resize(a, 9) + "00011"), 8);
a <= resize((resize(a, 9) + "00011"), 8);


Note: The above test with `signed = True` would fail early due to intbv limit checking.

### In-Simulation tracing

If the trace info was to be inserted into the simulation itself, we need a customized Checkpoint:

In [11]:
from myirl.kernel.sensitivity import LogicContext

class MyCheck(Checkpoint):
    def __call__(self, ctx = None):
        t = self.trace_info()
        # Set the message prefix:
        self._msg_pre = t + " : "      
        self.logic.clear() # When called multiple times
        self._create_logic(ctx)

In [12]:
from myirl.kernel.utils import LOG_VERBOSE, LOG_OFF

class MyBlockTrace(MyBlock2):
    
    check = MyCheck
    
    def worker(self, a, b):
        "MyIRL notation worker generator function"
        
        u = base.ConstSig(3, 5)
        v = base.ConstSig(2, 4)

        ma = b.set(a - u)
        mb = b.set(b + v) # Define a generator for this op
        print("--- WORKER ----")
        yield [ma]
        print("--- Procedural ----")
        l = []
        # Note: Inside, several references to the *same*
        # generator are casted. The generator must ensure
        # to allow multiple calls, keep in mind that some
        # generators append to their RTL body when called.
        
        # For traceback reasons, it is not recommended to create
        # the check as a generator:
        # c = self.check(b, "addition")
        for i in range(3):
            print(LOG_VERBOSE + "Iteration %d" % i + LOG_OFF)
            l += [mb, self.check(b, "addition%d" % i)]
        yield l
    

In [13]:
test(MyBlockTrace)

====================== MYHDL process ======================
a <= x"54";
b <= (a xor x"85");
-- Checkpoint macro::myhdl_worker1:13 : first check {
wait for 1 ns;
print("VAL" & " " & "0x"& hstr(b));
assert (b = x"d1")
    report "macro::myhdl_worker1:13 : first check -- expected: 209" severity failure;
-- }
>  macro::myhdl_worker1:13
b <= (resize(b, 9) + 1)(8-1 downto 0);
b <= resize(shift_right(resize(b, 8), 3), 8);
-- Checkpoint macro::myhdl_worker1:16 : second check {
wait for 1 ns;
print("VAL" & " " & "0x"& hstr(b));
assert (b = x"1a")
    report "macro::myhdl_worker1:16 : second check -- expected: 26" severity failure;
-- }
>  macro::myhdl_worker1:16
====================== MYIRL process ======================
--- WORKER ----
b <= resize((resize(a, 9) - "00011"), 8);
--- Procedural ----
Iteration 0
Iteration 1
Iteration 2
b <= resize((resize(b, 9) + x"2"), 8);
-- Checkpoint /tmp/ipykernel_21486/2690232996.py::worker:29 : addition0 {
wait for 1 ns;
print("VAL" & " " & "0x"& hstr(b));


## Issues

* For factorized code (AST-translated, i.e. MyHDL notation) as well as cythonized modules, the source file traceback may be not fully accurate or not possible for compiled code.
  * In rare cases, line numbers are off by one. This is a Python parsing issue with various
    Python interpreters
  * Not all code can be accessed correctly when nested in a class or factory function. Also a Python issue
  * Cython modules can only be traced back when source code and debug information is present
* Evaluation does not regard initial values of a signal. It always requires explicit assignment.